In [1]:
from dotenv import load_dotenv
load_dotenv('../dot.env')
import os
import getpass


def _set_if_undefined(var: str):
    # 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


# OPENAI_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("OPENAI_API_KEY")
# LANGCHAIN_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("LANGCHAIN_API_KEY")
# TAVILY_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "QA_SQL_CSV_Agents"

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [2]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent, create_json_chat_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

# llm = ChatOpenAI(
#     base_url = 'http://sionic.chat:8001/v1',
#     api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
#     model = 'xionic-ko-llama-3-70b',
#     temperature = 0.1,
# )
# # sql tool은 아래의 langchain.community.agent_toolkits.sql 사용
# # https://api.python.langchain.com/en/stable/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html#langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit

# # json prompt-ko -> 테디노트 작품
# json_prompt_ko = hub.pull('teddynote/react-chat-json-korean')
# # prompt 의 messages를 출력합니다.
# json_prompt_ko.messages

# toolkit = SQLDatabaseToolkit(db=db, llm=llm)
# # agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

# llama_3_agent = create_json_chat_agent(llm, tools = toolkit ,prompt= json_prompt_ko) #xionic-ko-llama-3-70b 사용
# agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, prompt=json_prompt_ko)
# llama3_agent_executor = AgentExecutor(agent=agent,
#                                         tools=toolkit,
#                                         verbose=True,
#                                         handle_parsing_errors = True,
#                                         return_intermediate_steps = True
#                                         )



response = agent_executor.invoke(
    {
        "input": "List the total sales per all countries. Which country's customers spent the most?"
    }
)
display(response)



> Entering new SQL Agent Executor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
print(response['output'])

Here are the total sales per country, with the country where customers spent the most listed first:

1. USA: $523.06
2. Canada: $303.96
3. France: $195.10
4. Brazil: $190.10
5. Germany: $156.48
6. United Kingdom: $112.86
7. Czech Republic: $90.24
8. Portugal: $77.24
9. India: $75.26
10. Chile: $46.62

The country where customers spent the most is the USA.


# 동적 퓨샷 프롬프트 사용
- 에이전트 성능을 최적화하기 위해, 도메인 특화 지식을 포함한 맞춤형 프롬프트를 제공할 수 있습니다. 
- 이 경우, 사용자 입력에 기반하여 동적으로 퓨샷 프롬프트를 구성하는 예시 선택기를 만들 것입니다. 
- 이는 모델이 참조할 수 있는 관련 쿼리를 프롬프트에 삽입함으로써 더 나은 쿼리를 생성하도록 도울 것입니다.
- 먼저 몇 가지 사용자 입력 SQL 쿼리 예시가 필요합니다: 


In [4]:
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

의미 검색 vector DB 생성
- 위에서 만든 예시 프롬프트를 바탕으로 
- 의미론적 검색을 하는 Vector DB 및 검색기 생성

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

APIConnectionError: Connection error.

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

NameError: name 'example_selector' is not defined

우리의 기본 에이전트는 OpenAI 도구 에이전트이며, OpenAI 함수 호출을 사용하기 때문에,
우리의 전체 프롬프트는 인간 메시지 템플릿과 에이전트 스크래치패드 MessagesPlaceholder를 포함한 채팅 프롬프트여야 합니다. few-shot 프롬프트는 우리 시스템 메시지로 사용될 것입니다:

In [10]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [11]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [13]:
agent.invoke({"input": "How many artists are there?"})['output']



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) AS TotalArtists FROM Artist'}`


[(275,)]There are 275 artists in the database.

> Finished chain.


'There are 275 artists in the database.'

# 고차원 칼럼 다루기
- 주소, 노래 이름 또는 아티스트와 같은 고유 명사를 포함하는 칼럼을 필터링하기 위해서는 데이터를 올바르게 필터링하기 위해 먼저 철자를 이중으로 확인해야 합니다.

- 데이터베이스에 존재하는 모든 고유 명사를 포함하는 벡터 저장소를 생성함으로써 이를 달성할 수 있습니다. 그런 다음 사용자가 질문에 고유 명사를 포함할 때마다 에이전트가 그 벡터 저장소를 조회하여 해당 단어의 올바른 철자를 찾을 수 있습니다. 이 방법으로 에이전트는 대상 쿼리를 구성하기 전에 사용자가 언급하는 엔티티를 이해할 수 있습니다.

- 먼저 원하는 각 엔티티에 대한 고유 값이 필요하며, 결과를 요소 목록으로 파싱하는 함수를 정의해야 합니다:


In [14]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Plays Metallica By Four Cellos',
 'Battlestar Galactica: The Story So Far',
 'Berlioz: Symphonie Fantastique',
 'Acústico MTV',
 'My Generation - The Very Best Of The Who']

In [15]:
from langchain.agents.agent_toolkits import create_retriever_tool

vector_db = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [17]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool! 

You have access to the following tables: {table_names}

If the question does not seem related to the database, just return "I don't know" as the answer."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"), MessagesPlaceholder("agent_scratchpad")]
)
agent = create_sql_agent(
    llm=llm,
    db=db,
    extra_tools=[retriever_tool],
    prompt=prompt,
    agent_type="openai-tools",
    verbose=True,
)

In [18]:
agent.invoke({"input": "How many albums does alis in chain have?"})['output']



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'alis in chain'}`


Alice In Chains

Aisha Duo

Xis

Da Lama Ao Caos

A-Sides
Invoking: `sql_db_query` with `{'query': "SELECT COUNT(*) AS album_count FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'Alice In Chains')"}`


[(1,)]Alice In Chains has 1 album.

> Finished chain.


'Alice In Chains has 1 album.'